In [1]:
from os.path import join
from glob import glob
import numpy as np
import dask
import dask.array as da
import pyklb as klb
import napari
import time

D:\Anaconda3\envs\denoising\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda3\envs\denoising\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\Anaconda3\envs\denoising\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
D:\Anaconda3\envs\denoising\lib\site-packages\napari\__init__.py:27: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.6. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [2]:
#Use opportunistic caching 
from dask.cache import Cache
cache = Cache(2e9)  # Leverage some memory
cache.register()    # Turn cache on globally

In [3]:
inPath = 'W:/SV3/RC_15-06-11/Dme_E2_His2AvRFP_spiderGFP_12-03_20150611_155054.corrected/Results/TimeFused/'
fnames = glob(join(inPath, '*/*CM00_CHN00.fusedStack.klb' ))
fnames.sort()

sample_full = klb.readfull(fnames[0]) #Sample image with klb.readfull
z= sample_full.shape[0]
y= sample_full.shape[1]
x= sample_full.shape[2]

In [4]:
# Segment each XY-image in the 3D stack is into a 3x3 grid

# X and Y strides to segment the each XY-image
y_stride= y/3
x_stride= x/3

ROIs_lobo = ([0,0,0],
             [0,y_stride,0],
             [0,2*y_stride,0],
             
             [0,0,x_stride],
             [0,y_stride,x_stride],
             [0,2*y_stride,x_stride],
             
             [0,0,2*x_stride],
             [0,y_stride,2*x_stride],
             [0,2*y_stride,2*x_stride]) 


ROIs_upbo = ([z-1,y_stride-1,x_stride-1],
             [z-1,2*y_stride-1,x_stride-1],
             [z-1,3*y_stride-1,x_stride-1],
             
             [z-1,y_stride-1,2*x_stride-1],
             [z-1,2*y_stride-1,2*x_stride-1],
             [z-1,3*y_stride-1,2*x_stride-1],
             
             [z-1,y_stride-1,3*x_stride-1],
             [z-1,2*y_stride-1,3*x_stride-1],
             [z-1,3*y_stride-1,3*x_stride-1]) 

sample = klb.readroi(fnames[0], ROIs_lobo[0], ROIs_upbo[0]) #Sample image with klb.readroi
print(sample.shape, sample.dtype)

(75, 256, 616) uint16


### Generate lazy arrays:

In [5]:
# For each ROI, lazily evaluate klb.readroi with dask.delayed 
lazy_values = [[dask.delayed(klb.readroi)(fn,ROIs_lobo[i], ROIs_upbo[i]) for fn in fnames] for i in range(len(ROIs_lobo))]
len(lazy_values)

9

In [6]:
#Construct lazy dask arrays for all lazy values
start = time.time()
lazy_arrays_0 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[0]]

lazy_arrays_1 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[1]]

lazy_arrays_2 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[2]]

lazy_arrays_3 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[3]]

lazy_arrays_4 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[4]]

lazy_arrays_5 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[5]]

lazy_arrays_6 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[6]]

lazy_arrays_7 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[7]]

lazy_arrays_8 = [da.from_delayed(lazy_value,
                               shape=sample.shape,
                               dtype=sample.dtype)
               for lazy_value in lazy_values[8]]

#Concatenate the ROIs
foo = da.concatenate([lazy_arrays_0, lazy_arrays_1, lazy_arrays_2], axis = 2)
fee = da.concatenate([lazy_arrays_3, lazy_arrays_4, lazy_arrays_5], axis = 2)
foos = da.concatenate([lazy_arrays_6, lazy_arrays_7, lazy_arrays_8], axis = 2)
a = da.concatenate([foo, fee, foos], axis = 3)

print(type(a), a.shape, a.dtype)
print('Image-data on disk:', 2*round(a.size/(1024**3),2), 'GB (uncompressed)')
print('Chunksize:', a.chunksize)
print('Took', round(time.time()-start,2), 'sec')

<class 'dask.array.core.Array'> (2701, 75, 768, 1848) uint16
Image-data on disk: 535.52 GB (uncompressed)
Chunksize: (1, 75, 256, 616)
Took 3.18 sec


###  Initialize the GUI first

In [7]:
%gui qt5

### Create a napari viewer and add the image as a layer

In [8]:
import napari
viewer = napari.Viewer(axis_labels = ['t','z','y','x'])
viewer.add_image(a,
                 scale = [1,8,1,1],
                 colormap='inferno',
                 blending='additive',
                 name='membrane',
                 is_pyramid = False,
                 rgb = False,
                 contrast_limits = [10,255])

<Image layer 'membrane' at 0x27e6281e808>